<a href="https://colab.research.google.com/github/TangoIndiaPapa/google-colab-test/blob/main/Colab_Python_Snippet_Secure_Secret_Handling_%26_Drive_Interaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# === Colab Cell 1: Mount Google Drive (If not already mounted) ===
# Purpose: This cell connects your Google Colab notebook to your Google Drive,
#          allowing the notebook to read from and write to files stored there.
#          It's like opening the main door to your workshop and connecting your workbench (Colab)
#          to your main parts storage (Google Drive).
# Who's calling it: You, "Me (Human)", by running this cell in Colab.
# Inputs required: User authorization via a pop-up window from Google.
# Logic it uses: Calls the `drive.mount()` function from the `google.colab` library.
# Outputs: Mounts Google Drive at '/content/drive'. Prints a confirmation message.
# Who's consuming the output: Subsequent cells that need to access '/content/drive'.
# Expected Output: You'll see a link to authorize, and then a message like:
#                  "Mounted at /content/drive"

from google.colab import drive # Import the 'drive' module from the 'google.colab' library
import os # Import the 'os' module for operating system dependent functionalities like path manipulation

try:
    # Attempt to mount Google Drive at the '/content/drive' path in the Colab environment.
    # The 'force_remount=True' can be useful if you need to ensure a fresh mount,
    # but often it's not needed if already mounted correctly. For a first run, it's fine.
    drive.mount('/content/drive', force_remount=True)

    # Print a success message confirming the mount point.
    print("Google Drive mounted successfully at /content/drive")
except Exception as e:
    # If any error occurs during mounting (e.g., user cancels authorization), print an error message.
    print(f"An error occurred while mounting Google Drive: {e}")
    # It's philosophically important to acknowledge when a foundational step fails.
    # Without the Drive, our workshop can't access its main storage.

# === Colab Cell 2: Define Base Project Path & Create Directory ===
# Purpose: This cell defines a standard base path within your Google Drive where all
#          projects generated or managed via this Colab notebook will reside.
#          It then creates this base directory if it doesn't already exist.
#          Think of this as designating a specific, well-organized area in your workshop
#          (Google Drive) for all "Gemini-assisted" projects.
# Who's calling it: You, "Me (Human)", by running this cell after successfully mounting Drive.
# Inputs required: None directly (uses the predefined path).
# Logic it uses: Uses `os.path.join` to construct the path and `os.makedirs` to create it.
# Outputs: Creates the directory on Google Drive if it's not there. Prints a confirmation.
# Who's consuming the output: Subsequent cells that will create project files under this base path.
# Expected Output: A message indicating whether the directory was created or already existed, e.g.,
#                  "Base directory for Gemini projects created: /content/drive/MyDrive/Gemini_Agent_Projects"
#                  or "Base directory already exists: /content/drive/MyDrive/Gemini_Agent_Projects"

# Assumption: Google Drive is successfully mounted at '/content/drive' from the previous cell.

# Define the root directory in your Google Drive for projects we work on.
# You can change "Gemini_Agent_Projects" to something else if you prefer,
# but ensure it's consistent. This is like naming the main department in your workshop.
GEMINI_PROJECTS_BASE_PATH = '/content/drive/MyDrive/Gemini_Agent_Projects'

# Check if the base directory exists, and create it if it doesn't.
# The 'exist_ok=True' argument for os.makedirs prevents an error if the directory already exists.
if not os.path.exists(GEMINI_PROJECTS_BASE_PATH): # Check if the path does not exist
    os.makedirs(GEMINI_PROJECTS_BASE_PATH, exist_ok=True) # Create directories, no error if it exists
    # Print a message confirming the creation of the base directory.
    print(f"Base directory for Gemini projects created: {GEMINI_PROJECTS_BASE_PATH}")
else:
    # Print a message indicating the base directory already exists.
    print(f"Base directory already exists: {GEMINI_PROJECTS_BASE_PATH}")

# Verify by listing the contents of MyDrive to see if the folder is there (optional check)
# This is like a quick visual inspection by the mechanic.
print("\nListing contents of your MyDrive (root) to help verify:")
try:
    # List only the first few items to avoid too much output if MyDrive is large.
    # This is a sanity check.
    drive_root_contents = os.listdir('/content/drive/MyDrive/')[:10] # Get first 10 items
    if drive_root_contents: # If there are contents
        for item in drive_root_contents: # Loop through items
            print(f"- {item}") # Print each item
    else:
        print("MyDrive appears to be empty or inaccessible for listing at the root.")
except FileNotFoundError: # Catch error if MyDrive path is somehow not found after mount
    print("Error: Could not list /content/drive/MyDrive/. Ensure Drive is correctly mounted.")
except Exception as e: # Catch any other unexpected errors
    print(f"An error occurred listing MyDrive contents: {e}")

# === Colab Cell 3: Instruction for Setting Up a Colab Secret ===
# Purpose: This cell DOES NOT RUN CODE. It provides instructions for YOU, the human,
#          to securely add a secret (like an API key) using Google Colab's built-in
#          Secrets manager. This is the most secure way to handle sensitive data
#          in Colab without hardcoding it into your notebook cells.
#          Think of this as the instruction manual for using the secure parts locker in your workshop.
#          You, the workshop owner, are the only one who puts the valuable key into this locker.
# Who's calling it: You read this and perform the action in the Colab UI.
# Inputs required: Your manual action in the Colab UI.
# Logic it uses: N/A (Informational text cell).
# Outputs: N/A (Guides your action).
# Who's consuming the output: You.
# Expected Output: This cell itself doesn't produce Python output. After you follow
#                  the instructions, a secret will be available to your Colab notebook runtime.

print("""
INSTRUCTIONS FOR SECURELY ADDING A SECRET:

1.  On the left sidebar of your Colab notebook, click the "Key" icon (🔑).
    This opens the "Secrets" pane.
2.  Click the "+ Add a new secret" button.
3.  For the 'Name' of the secret, enter exactly: MOCK_SENTRY_API_KEY
    (This name is case-sensitive and will be used by the next code cell to access it).
4.  For the 'Value' of the secret, enter any placeholder string for this demonstration,
    for example: my-super-secret-api-key-12345
    (In a real scenario, this would be your actual sensitive API key).
5.  Toggle the "Notebook access" switch ON for this secret. This allows this notebook
    to access the secret's value.
6.  You can leave "Grant access to all notebooks" OFF for better security unless you
    specifically need this secret in other notebooks.

Once you have added the secret MOCK_SENTRY_API_KEY with notebook access enabled,
you can proceed to run the next code cell (Cell 4).
""")

# === Colab Cell 4: Accessing the Colab Secret ===
# Purpose: This cell demonstrates how to securely access the secret that you (hopefully)
#          added in the previous step using Colab's Secrets manager.
#          The code itself never sees or stores the raw secret value directly if it's
#          managed properly by Colab; it just gets permission to use it.
#          This is like your authorized script getting the key from the secure locker
#          without knowing the locker's combination.
# Who's calling it: You, "Me (Human)", after setting up the secret as per Cell 3.
# Inputs required: The secret named 'MOCK_SENTRY_API_KEY' must be available via Colab Secrets.
# Logic it uses: Imports `userdata` from `google.colab` and uses `userdata.get()`
# Outputs: Prints a confirmation if the secret is found, or an error if not.
#          It will also print a masked version of the secret for demonstration.
# Who's consuming the output: You (to verify it worked), and potentially subsequent
#                           code that needs to use this API key.
# Expected Output: If successful:
#                  "Successfully accessed the secret named 'MOCK_SENTRY_API_KEY'."
#                  "Secret Value (Masked for security): my-s**********-12345" (example)
#                  If not found:
#                  "Error: Secret 'MOCK_SENTRY_API_KEY' not found or notebook access is not enabled."
#                  "Please ensure you've added it correctly using the Secrets (🔑) pane."

from google.colab import userdata # Import the 'userdata' module for accessing secrets

# Define the name of the secret we expect to find. This MUST match the name you used in Cell 3.
SECRET_NAME = 'MOCK_SENTRY_API_KEY' # The name of the secret

# Attempt to retrieve the secret value.
# userdata.get() is the secure way to access secrets in Colab.
try:
    # This is the moment our script requests the key from the secure locker.
    api_key_value = userdata.get(SECRET_NAME) # Get the secret value

    # Check if the secret was actually found and has a value.
    # Sometimes userdata.get() might return None if the secret doesn't exist or access is denied.
    if api_key_value is not None: # If the API key was successfully retrieved
        # Print a success message.
        print(f"Successfully accessed the secret named '{SECRET_NAME}'.")

        # For demonstration and security, let's print a masked version of the key.
        # Never print the full secret value in logs or production outputs!
        # This is like showing the mechanic that you have *a* key, but not revealing the actual cuts on it.
        if len(api_key_value) > 8: # If the key is long enough to mask meaningfully
            # Show first 4 chars, then asterisks, then last 5 chars.
            masked_key = f"{api_key_value[:4]}**********{api_key_value[-5:]}"
        elif len(api_key_value) > 0: # If it's short but not empty
            masked_key = "*" * len(api_key_value) # Mask the whole short key
        else: # If it's an empty string (should ideally not happen for a real key)
            masked_key = "(empty secret value)"
        print(f"Secret Value (Masked for security): {masked_key}")

        # Now, 'api_key_value' can be used by other parts of your script,
        # for example, to authenticate with an external service.
        # print("\nThis API key could now be used to initialize a client, for example:")
        # print("# from some_sdk import SomeServiceClient")
        # print(f"# client = SomeServiceClient(api_key='{masked_key}') # Using masked for print, actual would use api_key_value")

    else: # If userdata.get() returned None
        # This means the secret was not found or access was not granted.
        print(f"Error: Secret '{SECRET_NAME}' not found or notebook access is not enabled.")
        print("Please ensure you've added it correctly using the Secrets (🔑) pane and enabled 'Notebook access'.")
        api_key_value = None # Ensure it's None if not found

except Exception as e:
    # Catch any other exceptions that might occur (e.g., if userdata module itself has an issue).
    print(f"An error occurred while trying to access the secret '{SECRET_NAME}': {e}")
    print("Make sure you are running this in a Google Colab environment.")
    api_key_value = None # Ensure it's None on error

# === Colab Cell 5: Example File Operation Using Drive & "Secret" Context ===
# Purpose: This cell demonstrates a simple file operation (creating a dummy config/status file)
#          within our designated project directory on Google Drive. It also conceptually uses
#          the "API key" retrieved in the previous step (though just for inclusion in the file content here).
#          This simulates a part of what our "Intelligent DevOps Sentry" might do:
#          e.g., write a status report or a configuration file after using an API key.
#          It's like our mechanic, having fetched a special tool (API key), now uses it to
#          perform a task and makes a note in the project's logbook on the main shelf (Google Drive).
# Who's calling it: You, "Me (Human)", after successfully running Cells 1, 2, and 4.
# Inputs required:
#   - `GEMINI_PROJECTS_BASE_PATH` variable must be defined (from Cell 2).
#   - `api_key_value` variable should hold the secret value (from Cell 4) or be None.
# Logic it uses:
#   - Defines a dummy project name and path within GEMINI_PROJECTS_BASE_PATH.
#   - Creates this project directory if it doesn't exist.
#   - Writes a small text file into this project directory, including a reference to the API key's status.
# Outputs: Creates a directory and a file on Google Drive. Prints confirmation messages.
# Who's consuming the output: Your Google Drive, and you (verifying the file creation).
# Expected Output:
#   "Dummy project path: /content/drive/MyDrive/Gemini_Agent_Projects/SentryTestProject_v1"
#   (Message about creating SentryTestProject_v1 directory if it's new)
#   "Creating dummy status file: /content/drive/MyDrive/Gemini_Agent_Projects/SentryTestProject_v1/sentry_status_report.txt"
#   "Successfully created: /content/drive/MyDrive/Gemini_Agent_Projects/SentryTestProject_v1/sentry_status_report.txt"
#   "You can now check your Google Drive for this file!"
#   And if api_key_value was not found:
#   "Note: MOCK_SENTRY_API_KEY was not available for this report."

import datetime # Import the 'datetime' module for timestamping

# Assumption: GEMINI_PROJECTS_BASE_PATH is defined from Cell 2.
# Assumption: api_key_value is defined from Cell 4 (it might be None if the secret wasn't found).

# Define a name for a dummy test project where we'll create a file.
# This is like deciding on a specific car model we're working on in our designated project area.
dummy_project_name = "SentryTestProject_v1"
dummy_project_path = os.path.join(GEMINI_PROJECTS_BASE_PATH, dummy_project_name) # Construct full path

print(f"Dummy project path: {dummy_project_path}") # Print the path

# Create the dummy project directory if it doesn't exist.
# This is like setting up a specific bay in the workshop for this car model.
if not os.path.exists(dummy_project_path): # Check if the directory doesn't exist
    os.makedirs(dummy_project_path, exist_ok=True) # Create it
    print(f"Created dummy project directory: {dummy_project_path}") # Confirmation
else:
    print(f"Dummy project directory already exists: {dummy_project_path}") # Message if it exists

# Define the path and content for a dummy status file.
# This is the specific note or small part we're creating for this project.
status_file_name = "sentry_status_report.txt" # Name of the file
status_file_path = os.path.join(dummy_project_path, status_file_name) # Full path to the file
current_time_utc = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC") # Get current UTC time

# Prepare content for the file.
# We'll include whether our mock API key was found.
file_content = f"""# Sentry Agent Status Report
# Generated on: {current_time_utc}

This is a dummy report file created by the Colab snippet.
It demonstrates writing to your Google Drive within the designated project structure.

Project Path: {dummy_project_path}

Secret Access Status:
""" # Initial part of the file content

if api_key_value is not None: # If the API key was successfully retrieved in Cell 4
    file_content += "MOCK_SENTRY_API_KEY: Successfully accessed (value is kept secret, not written here).\n"
    file_content += "This indicates the Colab Secrets mechanism is working.\n"
    # In a real application, you would USE the api_key_value here, not write it.
    # e.g., client = SomeAPIClient(api_key=api_key_value)
    #       status = client.get_status()
    #       file_content += f"Actual API Status: {status}\n"
else: # If the API key was not found or an error occurred
    file_content += "MOCK_SENTRY_API_KEY: Not available or access failed.\n"
    file_content += "Please check Cell 3 instructions and Cell 4 output.\n"

file_content += "\nEnd of Report.\n" # End of file content

# Write the content to the file.
# This is the actual act of the mechanic writing the note and placing it in the car's logbook.
print(f"\nCreating dummy status file: {status_file_path}")
try:
    with open(status_file_path, 'w', encoding='utf-8') as f: # Open the file in write mode
        f.write(file_content) # Write the content
    print(f"Successfully created: {status_file_path}") # Confirmation message
    print("\nYou can now check your Google Drive for this file!") # User instruction
except Exception as e: # Catch any errors during file writing
    print(f"Error creating file {status_file_path}: {e}") # Error message

# You can also list the contents of the dummy project directory to verify.
# This is like the workshop manager double-checking the work.
print(f"\nContents of '{dummy_project_path}':")
try:
    project_contents = os.listdir(dummy_project_path) # List contents
    if project_contents: # If there are contents
        for item in project_contents: # Loop through items
            print(f"- {item}") # Print each item
    else:
        print("(Directory is empty or no files found)")
except Exception as e: # Catch errors
    print(f"Error listing contents of {dummy_project_path}: {e}")

Mounted at /content/drive
Google Drive mounted successfully at /content/drive
Base directory already exists: /content/drive/MyDrive/Gemini_Agent_Projects

Listing contents of your MyDrive (root) to help verify:
- MKTG580
- EMTM  MKTG 580.gdoc
- WriteUp Guidelines doc.gdoc
- Rohm and Haas.gdoc
- FTP - if record is unique.gdoc
- Green Cloud Inc Access.gdoc
- GC Business tracker.gsheet
- GC Task Tracker.gsheet
- service.gsheet
- Green Cloud Web Sharing.gdoc

INSTRUCTIONS FOR SECURELY ADDING A SECRET:

1.  On the left sidebar of your Colab notebook, click the "Key" icon (🔑).
    This opens the "Secrets" pane.
2.  Click the "+ Add a new secret" button.
3.  For the 'Name' of the secret, enter exactly: MOCK_SENTRY_API_KEY
    (This name is case-sensitive and will be used by the next code cell to access it).
4.  For the 'Value' of the secret, enter any placeholder string for this demonstration,
    for example: my-super-secret-api-key-12345
    (In a real scenario, this would be your actual se